In [ ]:

# my helper functions for this project
import importlib
import helper
importlib.reload(helper)


In [ ]:
# make the test data

import pandas as pd
import numpy as np

# positive class: 1 
# negative class: 0
# high score -> more likely to be positive

d = {'score': [0.05, 0.1 ,0.20, 0.70, 0.70, 0.90, 0.90, 0.95], 'label': [0,0, 1, 0, 1, 1, 0, 1]}
df = pd.DataFrame(data=d)
df




In [ ]:
# calculate TPR and FPR for each point on ROC space

importlib.reload(helper)
df = helper.calculateROCPoints(df)

df


In [ ]:

# plot roc curve
import matplotlib.pyplot as plt

cols = ['blue', 'green', 'orange', 'red', 'pink', 'lightgreen', 'red', 'lightgrey', 'lightgreen']

#df.interpolate(method='linear')
# df.plot(kind = 'line', x = 'fpr', y ='tpr')
# df.plot(kind = 'scatter', x = 'fpr', y ='tpr', c=cols)

plt.plot(df['fpr'], df['tpr'])
plt.scatter(df['fpr'], df['tpr'], c=cols, s=100)

plt.plot([0,0.5], [0.25,1], linestyle='--', color='green')
plt.xlabel('FPR')
plt.ylabel('TPR')
plt.show()



In [ ]:
import matplotlib.pyplot as plt 

# Calculate loss for different costs
nN = np.sum(df.label==0)
nP = np.sum(df.label==1)
piP = nP/(nP+nN)
piN = 1-piP

b = 2
c_0 = 1
c = c_0 / 2

fig, ax = plt.subplots()

#cols = ['blue', 'green', 'orange', 'darkgrey', 'red', 'pink', 'black', 'lightgreen', 'purple', 'brown', 'lightgrey']


# plot line in cost space for each F0 F1 pair in ROC space
for i in range(0,df.shape[0]):

    gradient = 2*(piP*df.loc[i,'tpr'] - piP*(1-df.loc[i,'fpr']))
    intercept = 2*piN * (1-df.loc[i,'tpr'])

    ax.axline((0, intercept), slope=gradient, color=cols[i])
    ax.set_xlim(0, 1)
    ax.set_ylim(0, 1)


# plot lower envelope
importlib.reload(helper)
lowerEnv = helper.lowerEnvelope(df)
plt.plot(lowerEnv['cost'], lowerEnv['loss'], color='black', linestyle=':')
ax.set_ylabel("Loss")
ax.set_xlabel("Cost")


plt.xlabel('Cost')
plt.ylabel('Loss')


In [ ]:

# brier curve where c = score

for i in range(1,df.shape[0]):
    df.loc[i,'brier_loss'] = 2*df.loc[i,'score']*pi_0*(1-df.loc[i,'f0']) + 2*(1-df.loc[i,'score'])*pi_1*df.loc[i,'f1']

df

In [ ]:
fig, ax = plt.subplots()
ax.set_xlim(0, 1)
ax.set_ylim(0, 1)
df.interpolate(method='linear').plot(kind = 'scatter', x = 'score', y ='brier_loss', ax=ax)


In [ ]:
# Decision curve analysis
# https://pmc.ncbi.nlm.nih.gov/articles/PMC2577036/

# choose a threshold probability where the expected benefit of treatment is equal to the expected benefit of avoiding treatment

#for c in np.arange(0, 1, 0.1):
#    print(c)

prevScore = -1
prevLabel = -1
for i in range(0,df.shape[0]):
    print(prevScore)
    #print(df[0:i])
    if ((prevScore != df.loc[i,'score']) | (prevLabel == df.label[i])):
        tp = (i - df[0:i].label.sum())
        fp = df[0:i].label.sum()
        print(tp, fp)
        n = 15#df.shape[0]
        #print(n)
        #df.loc[i, 'netben'] = tp/n - ((fp/n)*(df.loc[i,'score']/(1-df.loc[i,'score'])))
        #df.loc[i, 'netben_treatnone'] = 0/n - (0/n)*(df.loc[i,'score']/(1-df.loc[i,'score']))
        #df.loc[i, 'netben_treatall'] = n_0/n - (n_1/n)*(df.loc[i,'score']/(1-df.loc[i,'score']))
        dcaScore = 1 - df.loc[i,'score']
        df.loc[i, 'netben'] = tp/n - (fp/n)*(dcaScore/(1-dcaScore))
        df.loc[i, 'netben_treatnone'] = 0/n - (0/n)*(dcaScore/(1-dcaScore))
        df.loc[i, 'netben_treatall'] = n_0/n - (n_1/n)*(dcaScore/(1-dcaScore))
    else:
        df.loc[i, 'netben'] = np.nan
        
    prevScore = df.loc[i,'score']
    prevLabel = df.loc[i,'label']

#df.loc[0,'netben'] = np.nan
df.loc[0,'netben'] = 0
df.loc[15,'netben'] = np.nan

df


In [ ]:
fig, ax = plt.subplots()
ax.set_xlim(0, 1)
ax.set_ylim(-1, 1)
#df.interpolate(method='linear').plot(kind = 'line', x = 'score', y ='netben_treatall', ax=ax)

df['dcaScore'] = 1 - df.score
#dfnonan = df.dropna(subset='netben_treatall')
dfnonan = df.dropna(subset='netben')
dfnonan = dfnonan.set_index("dcaScore")
#dfnonan.reset_index(inplace=True)


#dfnonan.plot(kind = 'line', x = 'dcaScore', y ='netben_treatall', ax=ax)
#dfnonan.plot(kind = 'scatter', x = 'dcaScore', y ='netben', ax=ax, color='C2')

dfnonan.plot(y='netben', ax=ax, color='C2', marker='o')
dfnonan.plot(y='netben_treatall', ax=ax, marker='o')

#dfnonan.plot(kind = 'scatter', x = 'dcaScore', y ='netben', ax=ax, color='C2', use_index=False)
#dfnonan.plot(kind = 'line', x = 'dcaScore', y ='netben_treatall', ax=ax, use_index=False)
#plt.plot(df.dcaScore, df.netben_treatall, use_index=False)
#plt.scatter(x = df.dcaScore, y =df.netben)
ax.axline((0, 0), slope=0, color='C1')

print("prevalence:", n_0/n)


# plot vertical line for class distribution
ax.axline((pi_0, -1), (pi_0, 0), color='C6')

## TODO the treat all and treat none don't cross at the class distribution because of missing scores around that region.
## to fix this create a separate dataframe with uniform score values and generate the treat all values for that.



In [ ]:
print(df.to_string())